Maastricht_University_logo.svg

# Information Retrieval and Text Mining Course - Neural Reranking Tutorial
Authors: Abderrahmane Issam and Jan Scholtes

Version 2024-2025 V20250415

#Notebook 4

In this notebook we will learn how to use [Pyterrier](https://github.com/terrier-org/pyterrier), a Python framework that is built on top of the Java-based Terrier IR platform. Pyterrier can be used to index different formats of datasets and can be integrated with different models starting from classical approaches like BM25 up to neural models like ColBERT. We will learn how to use Pyterrier for indexing, search and evaluation.

## Setup

### If you have a local GPU (Do the following steps in your local env):
If you have a GPU and you want to run this notebook locally, then I suggest you set up a conda environement as follows:



```
conda create --name ir python=3.11.1 \\
conda install -c conda-forge openjdk=11 \\
pip install notebook
```
The second step is to start jupyter in your local machine as follows:
```
jupyter notebook \
    --NotebookApp.allow_origin='https://colab.research.google.com' \
    --port=8888 \
    --NotebookApp.port_retries=0
```
Then go to `connect to local runtime` (which you will find in the menu where you can change the runtime) and paste the jupter backend URL that tou got in the output of the previous command.

If you are running this locally, then you only need to install packages once, otherwise, you will need to install them at the start of the instance and restart the runtime when required to.

In [ ]:
!pip install python-terrier

In [ ]:
!pip install --upgrade "git+https://github.com/terrierteam/pyterrier_colbert.git" --use-pep517

## Indexing and Search Using Pyterrier

In [ ]:
!pip install faiss-gpu-cu12

import faiss
assert faiss.get_num_gpus() > 0

In [ ]:
# !pip install numpy --upgrade --force-reinstall

In [ ]:
import pyterrier as pt
if not pt.java.started():
    pt.init()

Pyterrier can create an index from different dataset formats including Pandas DataFrame which we demonstrate below. We will create a synthetic DataFrame of documents, as well as queries and qrels to use for evaluation. After indexing the documents, we create a BM25 model that we will use for search later.

In [ ]:
import pandas as pd

# 1. Define Documents
documents = pd.DataFrame([
    {"docno": "d1", "text": "PyTerrier is great for information retrieval."},
    {"docno": "d2", "text": "Terrier is a powerful information retrieval platform."},
    {"docno": "d3", "text": "Python is a popular programming language."},
    {"docno": "d4", "text": "This tutorial introduces PyTerrier basics."}
])

# 2. Define Queries
queries = pd.DataFrame([
    {"query": "information retrieval", "qid": "q1"},
    {"query": "programming tutorial", "qid": "q2"}
])

# 3. Define Relevance Judgments (qrels)
qrels = pd.DataFrame([
    {"qid": "q1", "docno": "d1", "label": 1},
    {"qid": "q1", "docno": "d2", "label": 0},
    {"qid": "q2", "docno": "d3", "label": 1},
    {"qid": "q2", "docno": "d4", "label": 1}
])

# 4. Indexing
index_path = "./index"
!rm -r "./index"    # Remove index if it exists

indexer = pt.index.DFIndexer(index_path)
index_ref = indexer.index(text=documents["text"], docno=documents["docno"])

# 5. Retrieval (BM25)
bm25 = pt.BatchRetrieve(index_ref, wmodel="BM25")

We can see the files were created:

In [ ]:
!ls -ltrh ./index

We can see statistics of our index as follows:

In [ ]:
index = pt.IndexFactory.of(index_ref)
print(index.getCollectionStatistics().toString())

In [ ]:
terms = index.getLexicon()

for entry in terms:
    print(entry.getKey())

### Exercise 1:
The number of terms (13) is less than the number of words in all the 4 documents. Explain the reason for this.

Answer here

These are the terms identified by Pyterrier.

In [ ]:
for kv in index.getLexicon():
  print("%s -> %s" % (kv.getKey(), kv.getValue().toString() ) )

We can search a Pyterrier index using BM25 with the following function:

In [ ]:
bm25.search("programming language")


### Exercise2

Define each column in the output of search above.

Answer here.

Pyterrier offers a way to evaluate models using a different metrics. The list of possible metrics to use is available here: https://pyterrier.readthedocs.io/en/latest/experiments.html#evaluation-measures-objects. \\

We can also see how `Experiment` accepts a DataFrame of queries and qrels which are used to compute the metrics.

In [ ]:
# 6. Evaluation Pipeline
pt.Experiment([bm25], queries, qrels, eval_metrics=["map", "P_10"], names=["BM25"])

### Exercise 3

Instead of BM25 use a TF_IDF model. Try using it to search for a query then pass it along with bm25 to `Experiment`.

In [ ]:
# Answer here

## Neural Reranking with ColBERT

Below we use `pyterrier_colbert` which is a plugin for Pyterrier that makes it possible to use a ColBERT model for indexing and retrieval. We will use the [Vaswani NPL corpus](http://ir.dcs.gla.ac.uk/resources/test_collections/npl/), a corpus of 11,429 scientific abstract, with corresponding queries and relevance assessments.

In [ ]:
!rm -rf ./colbertindex

import pyterrier_colbert.indexing

checkpoint="http://www.dcs.gla.ac.uk/~craigm/colbert.dnn.zip"

indexer = pyterrier_colbert.indexing.ColBERTIndexer(checkpoint, "./", "colbertindex", chunksize=3)
indexer.index(pt.get_dataset("irds:vaswani").get_corpus_iter())

In [ ]:
!ls -ltrh ./colbertindex

In [ ]:
pyterrier_colbert_factory = indexer.ranking_factory()

colbert_e2e = pyterrier_colbert_factory.end_to_end()

We search using ColBERT as follows. `% 5` is used to retrieve the top 5 most relevant entries.

In [ ]:
out = (colbert_e2e % 5).search("chemical reactions")
out

In [ ]:
out.loc[0, 'query_toks']

In [ ]:
out.loc[0, 'query_embs'].shape

### Exercise 4

There are two new columns in the search results: `query_toks` and `query_embs`. Explain what they are and explain the shape of `query_embs`.

Answer here.

In [ ]:
dataset = pt.datasets.get_dataset("vaswani")
index_path = "./index"

!rm -rf ./index
indexer = pt.TRECCollectionIndexer(index_path)

indexer = indexer.index(dataset.get_corpus())

bm25 = pt.BatchRetrieve(indexer, wmodel="BM25")

In the following code we create a sentence transformer reranker.

In [ ]:
import pandas as pd
from sentence_transformers import CrossEncoder

crossmodel = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L-6', max_length=512)

def _crossencoder_apply(df : pd.DataFrame):
  return crossmodel.predict(list(zip(df['query'].values, df['text'].values)))

cross_encT = pt.apply.doc_score(_crossencoder_apply, batch_size=128)

We create a reranking pipeline that starts with BM25 as a retriever, and ends with using a sentence transformer (cross_encT) for reranking the retrieved documents. The reranking step requires the document text as input, which is not returned by default by bm25, and that is why we add `pt.text.get_text(dataset, 'text')` to retrieve the text documents and add them to the output of BM25.

In [ ]:
dataset = pt.get_dataset('irds:vaswani')
cross_enc_rerank = bm25 >> pt.text.get_text(dataset, 'text') >> cross_encT

In [ ]:
out = (bm25 % 5).search("chemical reactions")
out

In [ ]:
out = (cross_enc_rerank % 5).search("chemical reactions")
out

The following demonstrates how we can contruct IR pipelines using Pyterrier. The pipeline starts by using bm25 to retrieve relevant documents, which are then used by QueryExpantion transformer which expands the query by adding informative terms that are collected from the relevant documents. The last part runs bm25 with the new query. This process is called Pseudo Relevance Feedback (PRF).

In [ ]:
query_expansion = bm25 >> pt.rewrite.QueryExpansion(indexer) >> bm25

In [ ]:
pt.Experiment(
    [bm25, query_expansion, colbert_e2e, cross_enc_rerank],
    dataset.get_topics(),
    dataset.get_qrels(),
    eval_metrics=["map", "P_10", "mrt"],
    names = ["BM25", "QE", "ColBERT", "BM25 >> CrossEnc"]
)

On this small dataset, we can see that BM25 achieves better results than ColBERT. Adding Query Expansion improves MAP a little. But using PRF with ColBERT hurts (ColBERT-PRF) the performance while slowing down the inference because of the extra PRF step.

### Exercise 5

Apply the same process as above on a dataset of your choice. You can find the list of datasets in Pyterrier here: https://pyterrier.readthedocs.io/en/latest/datasets.html#available-datasets

### Exercise 6

Add at least 2 other metrics and explain what each of them is trying to capture.

In [ ]:
# Answer both exercise 5 and 6 here

### Exercise 7

Follow the example in this [README](https://github.com/terrierteam/pyterrier_colbert/tree/ba5c86c0bc8da450dee361140541f35b5349a492) to implement Pseudo Relevance Feedback (PRF) with ColBERT. Describe how it works, and add ColBERT-PRF to `Experiment` to evaluate it against the other pipelines.

In [ ]:
# Answer here

### Exercise 8

Implement a reranking pipeline with ColBERT and add it to `Experiment`.

In [ ]:
# Answer here

## Arxiv Abstracts Retrieval

In [ ]:
!pip install setuptools==68.0.0       # you ca skip this if you are using a local instance

In [ ]:
!pip install arxiv==2.1.3

The following code will retrieve 1000 abstracts for the query "nlp".

In [ ]:
import arxiv

# Search for papers
search = arxiv.Search(
    query="nlp",
    max_results=1000,
    sort_by=arxiv.SortCriterion.SubmittedDate
)

We construct a dictionary of titles and abstracts.

In [ ]:
documents = []
for result in search.results():
    documents.append({
        'title': result.title,
        'abstract': result.summary,

    })

In [ ]:
documents[0]

Rerun this in case you had to restart the session.

In [ ]:
import pyterrier as pt
if not pt.java.started():
    pt.init()

To create an index, we need to to have a docno field which we create below, and a text field which is the abstract in this case. \\
We use `DFIndexer` which allows us to create by passing a list ids and text.

In [ ]:
import pandas as pd

index_path = './arxivindex'
!rm -r './arxivindex'

df = pd.DataFrame({
    'docno': ['doc'+str(i) for i in range(len(documents))],
    'text': [document['abstract'] for document in documents]
})

indexer = pt.DFIndexer(index_path)
indexer.index(docno=df.docno, text=df.text)

We create a BM25 retrieval model for the Arxiv index.

In [ ]:
bm25 = pt.BatchRetrieve(indexer, wmodel="BM25")

In [ ]:
(bm25 % 5).search("information retrieval")

In [ ]:
df[df['docno']=='doc670'].text.item()

### Exercise 9

Create a ColBERT index using the Arxiv documents retrieved above and use it to search for some queries. Try to highlight how it is different from BM25 through the query results. You can also change the query we used to get arxiv pages.